<img src="https://heig-vd.ch/docs/default-source/doc-global-newsletter/2020-slim.svg" alt="Logo HEIG-VD" style="width: 80px;" align="right"/>

# Cours APN - Labo 2 : Comparaison de méthodes <br>de groupement sur des données biomédicales

## Résumé

L'objectif de ce laboratoire est de comparer deux méthodes de groupement : _k_-moyennes et DBSCAN.  

Le jeu de données vient du domaine biomédical et possède une annotation de référence.  

Les tâches demandées sont les suivantes :
* préparer les données à partir de données brutes disponibles en ligne
* pour chaque méthode de groupement
  - décider quels sont les meilleurs hyper-paramètres par évaluation intrinsèque
  - avec ces paramètres, afficher aussi les scores par évaluation extrinsèque
  - visualiser les groupements en 2D
* comparer les deux méthodes de groupement et conclure.

In [1]:
# Nom et prénom : Muhlemann Julien & Ronquillo Cristhian

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import os
os.environ["OMP_NUM_THREADS"] = "1" # KMeans is known to have a memory leak ... set OMP_NUM_THREADS=1

import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN

from sklearn.metrics import davies_bouldin_score as dbs
from sklearn.metrics import silhouette_score as sis
from sklearn.metrics import adjusted_rand_score as ars
from sklearn.metrics import f1_score

from matplotlib.colors import Normalize # peut aider au coloriage des groupes (ou utiliser Seaborn)

## 1. Prise en main et analyse exploratoire des données

Les données biomédicales ont été présentées dans l'article de Todd R. Golub et al., [Molecular classification of cancer: class discovery and class prediction by gene expression monitoring](https://www.science.org/doi/abs/10.1126/science.286.5439.531), *Science*, 286:531-537, 1999.  Ces données représentent l'expression de **7129 gènes** dans des échantillons sanguins et de moëlle osseuse provenant de 72 patients souffrant de leucémie.  Pour chaque gène, son niveau d'expression est codé comme un nombre entier, positif ou négatif.  Le type de leucémie a été annoté pour chaque patient comme ALL ou AML (*acute lymphoblastic leukemia* ou *acute myeloid leukemia*) : ce sont les deux classes de référence.

Les données sont à télécharger depuis [une version fournie par C. Crawford sur Kaggle](https://www.kaggle.com/datasets/crawford/gene-expression) (pour information, une autre version se trouve dans le [package R golubEsets](https://master.bioconductor.org/packages/release/data/experiment/html/golubEsets.html)).  Si vous n'avez pas accès à Kaggle, le fichier `archive.zip` est fourni avec ce notebook.  Les données brutes contiennent :
* un fichier `train` avec les données de 38 patients : noms des gènes (longs et courts), niveau d'expression, signifiance ('call')
* un fichier `independent` avec les données de 34 autres patients
* un fichier `actual` qui indique pour chaque patient son type de leucémie (ALL ou AML)

**Le premier but est d'obtenir une DataFrame avec les 72 patients (lignes) et les niveaux des 7129 gènes (colonnes).  Le numéro de patient sert d'index.  Enfin, une colonne indiquera le type de chaque patient.**

a. Veuillez compléter le code ci-dessous pour transformer le premier fichier de données en un tableau:
* avec 7129 colonnes correspondant à la valeur exprimée de chaque gène pour chaque patient
* les noms des 7129 colonnes sont les noms courts de chaque gène (connus comme *Gene Accession Number*) 
* indexé par le numéro des patients, qui sera appelé 'Patient'

In [41]:
def conversion_donnees_brutes(f):
# Retourne la DataFrame selon la consigne.
# Code à compléter :
    df = pd.read_csv(f)
    #prendre les colonnes impaires en ignorant le 'call'
    expression_cols = df.columns[2::2]
    vrai_df = df[expression_cols].transpose()
    vrai_df.columns = df['Gene Accession Number']
    vrai_df.columns.name = 'Patient'
    vrai_df.index = vrai_df.index.astype(int)
    vrai_df = vrai_df.astype(int)
    return vrai_df    # Renommer les colonnes avec les noms des gènes


In [42]:
train_df = conversion_donnees_brutes(os.path.join('archive', 'data_set_ALL_AML_train.csv'))
print(train_df.shape) # vérifie le résultat : (38, 7129)
train_df.head(50)


(38, 7129)


Patient,AFFX-BioB-5_at,AFFX-BioB-M_at,AFFX-BioB-3_at,AFFX-BioC-5_at,AFFX-BioC-3_at,AFFX-BioDn-5_at,AFFX-BioDn-3_at,AFFX-CreX-5_at,AFFX-CreX-3_at,AFFX-BioB-5_st,...,U48730_at,U58516_at,U73738_at,X06956_at,X16699_at,X83863_at,Z17240_at,L49218_f_at,M71243_f_at,Z78285_f_at
1,-214,-153,-58,88,-295,-558,199,-176,252,206,...,185,511,-125,389,-37,793,329,36,191,-37
2,-139,-73,-1,283,-264,-400,-330,-168,101,74,...,169,837,-36,442,-17,782,295,11,76,-14
3,-76,-49,-307,309,-376,-650,33,-367,206,-215,...,315,1199,33,168,52,1138,777,41,228,-41
4,-135,-114,265,12,-419,-585,158,-253,49,31,...,240,835,218,174,-110,627,170,-50,126,-91
5,-106,-125,-76,168,-230,-284,4,-122,70,252,...,156,649,57,504,-26,250,314,14,56,-25
6,-138,-85,215,71,-272,-558,67,-186,87,193,...,115,1221,-76,172,-74,645,341,26,193,-53
7,-72,-144,238,55,-399,-551,131,-179,126,-20,...,30,819,-178,151,-18,1140,482,10,369,-42
8,-413,-260,7,-2,-541,-790,-275,-463,70,-169,...,289,629,-86,302,23,1799,446,59,781,20
9,5,-127,106,268,-210,-535,0,-174,24,506,...,356,980,6,177,-12,758,385,115,244,-39
10,-88,-105,42,219,-178,-246,328,-148,177,183,...,42,986,26,101,21,570,359,9,171,7


b. Veuillez appliquer la même fonction au deuxième fichier de données.

In [43]:
test_df = conversion_donnees_brutes(os.path.join('archive', 'data_set_ALL_AML_independent.csv'))
print(test_df.shape) # vérifier le résultat : (34, 7129)
test_df.head(10)

(34, 7129)


Patient,AFFX-BioB-5_at,AFFX-BioB-M_at,AFFX-BioB-3_at,AFFX-BioC-5_at,AFFX-BioC-3_at,AFFX-BioDn-5_at,AFFX-BioDn-3_at,AFFX-CreX-5_at,AFFX-CreX-3_at,AFFX-BioB-5_st,...,U48730_at,U58516_at,U73738_at,X06956_at,X16699_at,X83863_at,Z17240_at,L49218_f_at,M71243_f_at,Z78285_f_at
39,-342,-200,41,328,-224,-427,-656,-292,137,-144,...,277,1023,67,214,-135,1074,475,48,168,-70
40,-87,-248,262,295,-226,-493,367,-452,194,162,...,83,529,-295,352,-67,67,263,-33,-33,-21
42,22,-153,17,276,-211,-250,55,-141,0,500,...,413,399,16,558,24,893,297,6,1971,-42
47,-243,-218,-163,182,-289,-268,-285,-172,52,-134,...,174,277,6,81,2,722,170,0,510,-73
48,-130,-177,-28,266,-170,-326,-222,-93,10,159,...,233,643,51,450,-46,612,370,29,333,-19
49,-256,-249,-410,24,-535,-810,709,-316,27,14,...,76,1455,-123,491,-55,1950,906,79,170,-64
41,-62,-23,-7,142,-233,-284,-167,-97,-12,-70,...,129,383,46,104,15,245,164,84,100,-18
43,86,-36,-141,252,-201,-384,-420,-197,-60,-468,...,341,91,-84,615,-52,1235,9,7,1545,-81
44,-146,-74,170,174,-32,-318,8,-152,-148,17,...,180,690,-142,249,-220,354,-42,-100,45,-108
45,-187,-187,312,142,114,-148,-184,-133,12,97,...,37,125,-185,13,-148,304,-1,-207,112,-190


c. Veuillez fusionner les deux data frames en mettant les patients dans l'ordre 1 à 72.

In [45]:
# Veuillez écrire le code ici et appeler le résultat 'total_df'
total_df = pd.concat([train_df, test_df])
total_df.sort_index(inplace=True)


d.  En utilisant le fichier `actual.csv`, veuillez ajouter à **total_df** une colonne intitulée **type**, qui indique pour chaque patient son type de maladie (ALL ou AML).  Attention, cette colonne servira uniquement pour l'évaluation extrinsèque en fin de labo.

In [48]:
# Veuillez écrire le code ici :
patients = pd.read_csv(os.path.join('archive', 'actual.csv'))
patients.sort_index(inplace=True) #trier pour fusionner les mêmes patients
total_df["type"] = patients["cancer"]
total_df.head()

Patient,AFFX-BioB-5_at,AFFX-BioB-M_at,AFFX-BioB-3_at,AFFX-BioC-5_at,AFFX-BioC-3_at,AFFX-BioDn-5_at,AFFX-BioDn-3_at,AFFX-CreX-5_at,AFFX-CreX-3_at,AFFX-BioB-5_st,...,U58516_at,U73738_at,X06956_at,X16699_at,X83863_at,Z17240_at,L49218_f_at,M71243_f_at,Z78285_f_at,type
1,-214,-153,-58,88,-295,-558,199,-176,252,206,...,511,-125,389,-37,793,329,36,191,-37,ALL
2,-139,-73,-1,283,-264,-400,-330,-168,101,74,...,837,-36,442,-17,782,295,11,76,-14,ALL
3,-76,-49,-307,309,-376,-650,33,-367,206,-215,...,1199,33,168,52,1138,777,41,228,-41,ALL
4,-135,-114,265,12,-419,-585,158,-253,49,31,...,835,218,174,-110,627,170,-50,126,-91,ALL
5,-106,-125,-76,168,-230,-284,4,-122,70,252,...,649,57,504,-26,250,314,14,56,-25,ALL


e. Y a-t-il des données manquantes dans la Data Frame obtenue ?

f. Y a-t-il des *outliers* dans chaque colonne ?  Même s'il y en a, ne pas les supprimer.

g. Veuillez donner un résumé pour chaque colonne numérique (le _5-number-summary_) en utilisant la méthode _describe()_.  Utiliser l'affichage par défaut qui affiche un petit nombre de colonnes.

h. Combien de fois apparaît chaque classe de référence, ALL et AML ?

i. Appliquez la PCA sur des données normalisées (sans la colonne `type`), en **deux** dimensions (sans normaliser le résultat). **Vous travaillerez sur ces données tout au long du labo.**

In [ ]:
# Veuillez nommer X_pca le résultat de la PCA sur les colonnes de données.
# Veuillez nommer Y la colonne indiquant le type correct de chaque patient.


k. À ce stade, après la PCA, les points vous semblent-ils présenter un groupement visible ?

## 2. Groupement avec _k_-moyennes

Dans cette section, on vous demande de grouper les patients en utilisant la méthode des _k_-moyennes (avec initialisation _kmeans++_) et de trouver la valeur optimale du nombre de groupes entre 2 et 12, en utilisant l'indice de Davies-Bouldin et le score Silhouette.  Vous *ne* devez *pas* utiliser l'information du type de maladie.

Vu le nombre élevé d'attributs, vous réutiliserez les résultat de l'ACP faite ci-dessus, sans normalisation après l'ACP.

a. Pour les trois scores suivants, veuillez rappeler (en utilisant la documentation de sklearn) quel est leur intervalle de valeurs et quelles valeurs indiquent un meilleur groupement :
* l'indice de Davies-Bouldin
* le coefficient de Silhouette
* l'inertie.

In [ ]:
# Veuillez répondre ici.


b. Pour un nombre de groupes _k_ allant de 2 à 12 compris, veuillez effectuer le groupement avec _k_-moyennes, puis représenter sur trois graphiques la variation des trois scores intrinsèques en fonction de _k_.  Utiliser `n_init=10`.

c. Selon les courbes affichées, quelle valeur de _k_ proposez-vous de retenir ?  Veuillez donner une raison.

Si vous exécutez plusieurs fois l'affichage des courbes, la valeur optimale de _k_ reste-t-elle la même ?  Que pouvez-vous en conclure ?

Note : si vous fixez l'attribut `random_seed` de KMeans, alors les courbes ne varient pas au fil des exécutions, sauf si vous changez `n_init`.

In [ ]:
# Votre discussion ici.

d. Avec la valeur de _k_ choisie au (c), veuillez calculer deux scores extrinsèques : l'indice de Rand ajusté et le score F1.  Veuillez commenter brièvement ces scores.

_Indication pour calculer le F1-score, qui n'est pas implémenté dans sklearn pour le groupement._  Pour chaque patient, assignez-lui le type qui est majoritaire dans son groupe (ALL ou AML) en utilisant les bonnes réponses.  Puis comparez les types ainsi assignés avec les types corrects en utilisant directement le score F1 de sklearn.  Notez que pour simplifier, cette méthode est un peu différente de celle du cours.

In [ ]:
# Effectuer le groupement k-moyennes avec la meilleure valeur de k retenue au (b).


In [ ]:
# Calcul et affichage des deux scores.


In [ ]:
# Votre commentaire sur les scores.

e. Veuillez représenter les points avec PCA en 2D, en gardant les marqueurs précédents pour les deux types (cercle et triangle), et en utilisant des couleurs pour coder les groupes trouvés. Suggestion : réutiliser le code du Labo 1.  Veuillez commenter le résultat.

In [ ]:
# Utiliser le groupement obtenu ci-dessus.


In [ ]:
# Veuillez écrire ici votre discussion des résultats, à l'aide du graphique.


## 3. Groupement avec DBSCAN

Dans cette section, vous répondrez aux mêmes questions que dans la section précédente, mais pour la méthode DBSCAN, en utilisant les mêmes mesures intrinsèques que ci-dessus (sauf l'inertie).

a. Veuillez représenter sur des _heatmaps_ la variation des deux scores intrinsèques et le nombre de groupes en fonction des paramètres _eps_ et _min_samples_ :
* chercher _eps_ entre 0.5 et 14 par sauts de 0.5
* chercher _min_samples_ entre 1 et 12

In [ ]:
# Obtenir les deux scores et le nombre de groupes en variant les deux paramètres, les mettre dans des tableaux 2D


In [ ]:
# Afficher les trois heatmaps : Davies_Bouldin, Silhouette, et Nombre_de_groupes


b. Selon les scores trouvés, quelles valeurs de _eps_ et _min_samples_ proposez-vous de retenir ?  Pourquoi ?

Attention : les trois heatmaps ne sont pas toujours en accord, donc vous avez une certaine liberté pour choisir les deux paramètres (il n'y a pas d'optimum évident).

In [ ]:
# Votre discussion ici.

c. Avec les valeurs choisies au (b), veuillez calculer les deux scores extrinsèques comme dans la partie (2).

In [ ]:
# Effectuer le groupement DBSCAN avec les meilleurs paramètres,
# puis calculer l'indice de Rand ajusté et le score F1.


In [ ]:
# Votre commentaire sur les scores.

d. Veuillez représenter les points en 2D avec la même représentation que ci-dessus : utilisez des marques différentes pour les deux types, et des couleurs pour coder les groupes trouvés.  Veuillez commenter le résultat.

In [ ]:
# Veuillez commenter le graphique ici.


## 4. Conclusion

Comment se comparent les deux méthodes optimisées, en termes de scores extrinsèques ?  Veuillez discuter les paramètres trouvés et les scores obtenus.  En particulier, y a-t-il des sous-types des leucémies ALL et AML qui sont visibles sur les groupements que vous obtenus ?  Combien pour chacun ?

In [ ]:
# Votre discussion ici.

***
**Fin du Labo 2.**  Veuillez nettoyer ce notebook en gardant seulement les réponses et résultats désirés, l'enregistrer en remplaçant 'student' par votre nom, et le soumettre sur Cyberlearn.